In [1]:
import psycopg2
import boto3
import pandas as pd
from sqlalchemy import create_engine

host = 'database-sparkathon.cm6vbnnsye5g.us-east-1.rds.amazonaws.com'
port = 5432
database = 'postgres'
user = 'postgres'
password = "1QCF03Azp9wUQ3EfYeqI"

engine = create_engine(
    f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}",
    connect_args={"sslmode": "require"}
)

In [3]:
sp_core_base_query = """
select *
from
data_dev.sp_komodo_core_base
"""

sp_core_df = pd.read_sql(sp_core_base_query, engine)
sp_core_df.head()

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA
1,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA
2,1.003222e+09,5280135280,WI,M,5,88ZFPLSE,M,5,WI
3,1.003222e+09,5280135280,WI,M,5,Y57KB31F,M,4,WI
4,1.003222e+09,5280135280,WI,M,5,2XJVDDQ4,M,4,WI


In [5]:
sp_query = """
select *
from
data_dev.sp_data
"""

sp_data_df = pd.read_sql(sp_query, engine)
sp_data_df.head()

,patient_id,referral_date,patient_gender,patient_age,patient_type,rx_id,rx_written_date,ship_date,hub_enrollment_status,enrollment_number,...,final_patient_status,final_case_type,is_dummy_row,record_id,prescriber_zip,prescriber_npi,reopen_flag,bpg_alert_flag,mcoid_mmit,mconame_mmit
0,79540079,2025-08-12,M,31,Adult,NaN,NaT,45895,None,67,...,Active,Commercial,No,PN288502530856-9300,20889.0,1.073725e+09,N,N,196.0,TRICARE
1,79540079,2025-08-12,M,31,Adult,1.900610e+13,2025-08-12,2025-09-04 00:00:00,None,67,...,Active,Commercial,No,PN288554630363-9300,20889.0,1.073725e+09,N,N,196.0,TRICARE
2,79540079,2025-08-12,M,31,Adult,1.900610e+13,2025-08-12,2025-10-03 00:00:00,None,67,...,Active,Commercial,No,PN288796330363-9300,20889.0,1.073725e+09,N,N,196.0,TRICARE
3,79540079,2025-08-12,M,31,Adult,1.900610e+13,2025-08-12,2025-10-30 00:00:00,None,67,...,Active,Commercial,No,PN290152420210-9300,20889.0,1.073725e+09,N,N,196.0,TRICARE
4,79540079,2025-08-12,M,31,Adult,1.900610e+13,2025-08-12,2025-09-05 00:00:00,None,67,...,Active,Commercial,No,PN290528880910-9300,20889.0,1.073725e+09,N,N,196.0,TRICARE


In [6]:
mx_query = """
select *
from
data_dev.mx_core_table
"""

mx_df = pd.read_sql(mx_query, engine)
mx_df.head()

,medical_event_id,patient_id,service_date,service_to_date,procedure_code,procedure_code_type,modifiers,ndc_11,units,unit_type,...,revenue_code,bill_type_code,visit_type,kh_plan_id,diagnosis_codes,event_source,billing_npi_confidence,run_id,created_at,updated_at
0,663e4354f016ae553d173c57976ebe21eb4286ec516f26...,WLC2PZ7D,2024-05-01,2024-05-01,None,None,None,None,0,None,...,0258,111,INPATIENT,7286,|G40909|C712|F1010|F418|F1290|F17200|Z86711|Z9...,INSTITUTIONAL,A - KNOWN,None,None,None
1,975fd956c2aa3b3390293031b5650f1a41f70a77d0edfe...,JXP5L9HT,2023-09-05,2023-09-05,None,None,None,None,1,None,...,0430,117,INPATIENT,8975,|D320|G936|G935|D649|F32A|R001|D72829|N939|T38...,INSTITUTIONAL,A - KNOWN,None,None,None
2,d97f25fe58b765311ed1a948204336f6972277f97459c8...,VBJQD5WM,2024-10-03,2024-10-03,83735,CPT,None,None,None,None,...,None,131,OUTPATIENT,9378,|G9389|R531|Z4682|Z4659|G919|,INSTITUTIONAL,A - KNOWN,None,None,None
3,894c9a71a02083611e2001bb446344cc1cd9ce93738dca...,BR92EMW8,2024-09-12,2024-09-12,97110,CPT,|GP|,None,1,None,...,None,None,OUTPATIENT,6109,|D496|C717|M6281|R260|,PROFESSIONAL,A - KNOWN,None,None,None
4,bfea0b3b111bb3c342cddfe93365959bbf5d1109eb1ddf...,B1K0VVWZ,2024-09-30,2024-09-30,70553,CPT,|26|GC|,None,10,None,...,None,None,OUTPATIENT,7653,|G911|G914|,PROFESSIONAL,A - KNOWN,None,None,None


DIAGNOSIS FUNCTIONS AND FLAG BUILDING

In [ ]:
def get_icd_code_from_data(patient_id, sp_data):
    icd_codes_per_patient: list[str] = (
                                            sp_data.loc[sp_data['patient_id'] == patient_id, 'primary_icd_code']
                                            .dropna()
                                            .unique()
                                        )

    for code in icd_codes_per_patient:
        if code.startswith('C71'):
            return code.replace('.', '')

sp_core_df['icd_code'] = sp_core_df['sp_patient_id'].apply(get_icd_code_from_data, args=(sp_data_df,))

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state,icd_code,icd_code_claims,parent_diag_code_flag,exact_diag_code_flag
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA,C719,C719,1,1
1,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA,C719,C719,1,1
2,1.003222e+09,5280135280,WI,M,5,88ZFPLSE,M,5,WI,C719,C719,1,1
3,1.003222e+09,5280135280,WI,M,5,Y57KB31F,M,4,WI,C719,C719,1,1
4,1.003222e+09,5280135280,WI,M,5,2XJVDDQ4,M,4,WI,C719,C719,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2380,1.992402e+09,1880171880,VA,M,12,06R2J3C9,M,13,VA,C719,C719,1,1
2381,1.992402e+09,1880171880,VA,M,12,S9E2PSB7,M,13,VA,C719,C719,1,1
2382,1.992402e+09,1880171880,VA,M,12,38B8371E,M,13,VA,C719,C719,1,1
2383,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA,C719,None,0,0


In [94]:
import re

def get_icd_code_from_claims(row, claims):
    patient_id = row['matched_patient_id']
    sp_diag_code = row['icd_code']

    if pd.isna(patient_id) or pd.isna(sp_diag_code):
        return None

    sp_diag_code = str(sp_diag_code)
    parent = sp_diag_code[:3] 

    dx_values = (
        claims.loc[claims['patient_id'] == patient_id, 'diagnosis_codes']
        .dropna()
        .unique()
    )

    for dx in dx_values:
        dx_str = str(dx)
        if sp_diag_code in dx_str:
            return sp_diag_code

    pattern = re.compile(rf"{re.escape(parent)}(?:\.\d+|\d+)?")
    for dx in dx_values:
        dx_str = str(dx)
        m = pattern.search(dx_str)
        if m:
            return m.group(0).replace('.', '')

    return None

sp_core_df['icd_code_claims'] = sp_core_df.apply(
    get_icd_code_from_claims,
    axis=1,
    args=(mx_df,)
)

In [95]:
def generate_diagnosis_flags(row):
    exact_sp_diag_code = row['icd_code']
    exact_claims_diag_code = row['icd_code_claims']

    if pd.isna(exact_claims_diag_code) or pd.isna(exact_sp_diag_code):
        return 0, 0

    parent_sp_diag_code = str(exact_sp_diag_code)[:3]
    parent_claims_diag_code = str(exact_claims_diag_code)[:3]

    exact_match = 1 if exact_sp_diag_code == exact_claims_diag_code else 0
    parent_match = 1 if parent_sp_diag_code == parent_claims_diag_code else 0

    return parent_match, exact_match

sp_core_df[['parent_diag_code_flag', 'exact_diag_code_flag']] = sp_core_df.apply(
    generate_diagnosis_flags,
    axis=1,
    result_type='expand'
)
    

In [129]:
def calculate_diag_freq(row, mx_df):
    claims_patient_id = row['matched_patient_id']
    exact_diag_code_flag = row['exact_diag_code_flag']
    icd_code_claims = row['icd_code_claims']

    freq = 0

    if exact_diag_code_flag == 1 and pd.notna(claims_patient_id) and pd.notna(icd_code_claims):
        filtered_df = mx_df.loc[mx_df['patient_id'] == claims_patient_id]
        icd_codes = filtered_df['diagnosis_codes'].dropna().astype(str).values

        target = str(icd_code_claims)
        for code in icd_codes:
            if code.find(target) != -1:
                freq += 1

    return freq

freqs = []
for _, row in sp_core_df.iterrows():
    freqs.append(calculate_diag_freq(row, mx_df))

sp_core_df['diag_freq'] = freqs

In [155]:
from pandas import DataFrame
from datetime import datetime, timedelta

def generate_diagnosis_lookback(row, sp_data: DataFrame, mx_df):
    sp_patient_id = row['sp_patient_id']
    claims_patient_id = row['matched_patient_id']
    icd_code = row['icd_code']
    exact_diag_code_flag = row['exact_diag_code_flag']

    if exact_diag_code_flag != 1 or pd.isna(sp_patient_id) or pd.isna(claims_patient_id) or pd.isna(icd_code):
        return 0, 0, 0
    
    sp_ref_dates = pd.to_datetime(
        sp_data.loc[sp_data['patient_id'] == sp_patient_id, 'referral_date'],
        errors='coerce'
    )

    max_referral_date = sp_ref_dates.max()

    if pd.isna(max_referral_date):
        return 0, 0, 0

    date_90_days_before = max_referral_date - timedelta(days=90)
    date_180_days_before = max_referral_date - timedelta(days=180)
    date_360_days_before = max_referral_date - timedelta(days=360)

    claims = mx_df.loc[mx_df['patient_id'] == claims_patient_id].copy()
    claims['service_date'] = pd.to_datetime(claims['service_date'], errors='coerce')

    claims = claims.dropna(subset=['service_date'])

    claims_before_90_days = claims.loc[claims['service_date'] >= date_90_days_before]
    claims_before_180_days = claims.loc[claims['service_date'] >= date_180_days_before]
    claims_before_360_days = claims.loc[claims['service_date'] >= date_360_days_before]

    icd_code_str = str(icd_code)

    mask_90 = claims_before_90_days['diagnosis_codes'].astype(str).str.contains(icd_code_str, na=False)
    mask_180 = claims_before_180_days['diagnosis_codes'].astype(str).str.contains(icd_code_str, na=False)
    mask_360 = claims_before_360_days['diagnosis_codes'].astype(str).str.contains(icd_code_str, na=False)

    return int(mask_90.sum()), int(mask_180.sum()), int(mask_360.sum()) 

sp_core_df[['90_days_lookback_freq', '180_days_lookback_freq', '360_days_lookback_freq']] = sp_core_df.apply(
    generate_diagnosis_lookback,
    axis=1,
    args=(sp_data_df, mx_df),
    result_type='expand'
)

In [170]:
def generate_lookback_frequency(row, mx_df):
    icd_code = row['icd_code']
    exact_diag_code_flag = row['exact_diag_code_flag']
    claims_patient_id = row['matched_patient_id']

    if not (exact_diag_code_flag == 1 and pd.notna(icd_code) and pd.notna(claims_patient_id)):
        return None

    claims_filtered = mx_df.loc[
                    (mx_df['patient_id'] == claims_patient_id) & 
                    (mx_df['diagnosis_codes']
                    .astype(str)
                    .str
                    .contains(icd_code, na=False))].copy()

    if claims_filtered.empty:
        return None

    claims_filtered['service_date'] = pd.to_datetime(claims_filtered['service_date'], errors='coerce')
    claims_filtered = claims_filtered.dropna(subset=['service_date'])

    first_claim = claims_filtered['service_date'].min()
    last_claim = claims_filtered['service_date'].max()

    time_period = (last_claim - first_claim).days

    return int(time_period)

sp_core_df['first_last_claim_lookback'] = sp_core_df.apply(
    generate_lookback_frequency,
    axis=1,
    args=(mx_df,)
)

THERAPY AREA FUNCTIONS AND FLAG BUILDING

In [185]:
treatment_codes = {
    "radiation": [
        "61796","61797","61798","61799","61800",
        "77261","77262","77263","77280","77285","77290","77293","77295",
        "77300","77301","77306","77307","77316","77317","77318",
        "77321","77331","77332","77333","77334","77336","77338",
        "77370","77371","77372","77373","77385","77386","77387",
        "77401","77402","77407","77412","77417","77424","77425",
        "77427","77431","77470",
        "77520","77522","77523","77525",
        "77761","77762","77763","77767","77768","77770","77771","77772",
        "0395T",
        "G0339","G0340",
        "G6001","G6002","G6003","G6004","G6005","G6006","G6007",
        "G6008","G6009","G6010","G6011","G6012","G6013","G6014",
        "G6015","G6016","G6017",
        "D0012ZZ","D0Y07ZZ","D0010ZZ","D0000ZZ","D0002ZZ","D0001ZZ",
        "D0Y0FZZ","D0Y17ZZ","D0006ZZ","D0011ZZ","D0003ZZ","D0005ZZ"
    ],

    "chemotherapy": [
        "96400","96408","96412","96414","96445","96545","99555",
        "96360","96361","96365","96366","96367","96368","96369",
        "96370","96371","96372","96373","96374","96375","96376","96379",
        "96409","96411","96413","96415","96417","96549",
        "G0498","G8928","G8929",
        "J8999",
        "J9000-J9999",   # range indicator (handle separately in logic)
        "Q0081","Q0084"
    ],

    "surgery": [
        "61790","81463",
        "61750","61751",
        "61510","61518","61519","61520","61512","61521",
        "61526","61530"
    ]
}


In [197]:
sp_df = pd.read_sql("select * from data_dev.sp_komodo_core_base", engine)
sp_df

,sp_prescriber_npi,sp_patient_id,sp_state,sp_gender,sp_age,matched_patient_id,matched_gender,matched_age_2025,matched_state
0,1.003101e+09,9279339279,MA,F,64,9MZ6V2LE,F,64,MA
1,1.003101e+09,9279339279,MA,F,64,0XSZNET0,F,63,MA
2,1.003222e+09,5280135280,WI,M,5,88ZFPLSE,M,5,WI
3,1.003222e+09,5280135280,WI,M,5,Y57KB31F,M,4,WI
4,1.003222e+09,5280135280,WI,M,5,2XJVDDQ4,M,4,WI
...,...,...,...,...,...,...,...,...,...
2380,1.992402e+09,1880171880,VA,M,12,06R2J3C9,M,13,VA
2381,1.992402e+09,1880171880,VA,M,12,S9E2PSB7,M,13,VA
2382,1.992402e+09,1880171880,VA,M,12,38B8371E,M,13,VA
2383,1.992402e+09,1880171880,VA,M,12,JJSW263R,M,13,VA


In [ ]:
def generate_treatment_flag(row, treatment_codes: list[str], mx: DataFrame):
    mx_patient_id = row['matched_patient_id']

    if pd.isna(mx_patient_id):
        return 0, 0

    codes = [str(c) for c in treatment_codes]

    mx_filtered = mx.loc[
        (mx['patient_id'] == mx_patient_id) &
        (mx['procedure_code'].astype(str).isin(codes))
    ]

    if mx_filtered.empty:
        return 0, 0

    return 1, int(len(mx_filtered))

In [208]:
for treatment, codes in treatment_codes.items():
    sp_df[[f'{treatment}_flag', f'{treatment}_freq']] = sp_df.apply(
        generate_treatment_flag,
        axis=1,
        args=(codes, mx_df),
        result_type='expand'
    )

In [227]:
sp_df.to_excel(r'therapy.xlsx')

In [261]:
sp_core_df.drop_duplicates(subset=('sp_prescriber_npi', 'sp_patient_id', 'matched_patient_id'), inplace=True)
sp_df.drop_duplicates(subset=('sp_prescriber_npi', 'sp_patient_id', 'matched_patient_id'), inplace=True)

In [262]:
keys = ['sp_prescriber_npi', 'sp_patient_id', 'sp_state', 'sp_gender', 'sp_age', 'matched_patient_id', 'matched_gender', 'matched_age_2025', 'matched_state']

final_df = pd.merge(sp_core_df, sp_df, how='inner', on=keys)

In [265]:
final_df.to_excel('final_diagnosis_therapy.xlsx', index=False)